In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess
import re

In [ ]:
serial_data = []

In [ ]:
# only season 3 has legitimate links
# parses transcript links
class serial(scrapy.Spider):
    name = 'serial'
    
    def start_requests(self):
        url = 'https://serialpodcast.org/'
        yield scrapy.Request(url=url, callback=self.parse_episode)
        
    def parse_episode(self, response):
        episode_links = response.xpath('//div[contains(@class,node-episode)]//a[@class="card clearfix"]/@href').extract()
        for link in episode_links:
            link = 'https://serialpodcast.org' + link
            yield response.follow(url=link, 
                                  callback =self.parse_transcript_page) 
    
    def parse_transcript_page(self, response):
        transcript_links = response.xpath('//a[@class="transcript"]/@href').extract()
        for link in transcript_links:
            link = 'https://serialpodcast.org' + link
            yield response.follow(url=link,
                                  callback=self.parse_transcript)
    
    def parse_transcript(self, response):
        current_ep = []
        
        title = response.xpath('//header/h1/text()').extract()[0].split(':')
        current_ep.append(title[0].split(' ')[1])
        current_ep.append(title[1]) 
        # no date
        
        text = response.xpath('//div[@class="field-item even"]/*//text()').extract()
        text = '  '.join(text)
        current_ep.append(text)
        serial_data.append(current_ep)

In [ ]:
process = CrawlerProcess()
process.crawl(serial)
process.start()

In [ ]:
import pandas as pd

serial_df = pd.DataFrame(serial_data,
                         columns = ['Episode','Title','Text'])
serial_df

In [ ]:
serial_df.Text[0]
# can pick out hosts by finding the most common bigrams